# TSDAE Embeddings

## Workspace Setup

In [ ]:
!pip install sentence-transformers
!pip install datasets
!pip install pylatexenc

In [ ]:
# Neccessary imports

import json
import pandas as pd
import math
import os
import gzip
import csv
import random
import time

from pylatexenc.latex2text import LatexNodes2Text

import nltk
nltk.download('punkt')

from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

from sentence_transformers import models, util, evaluation, losses
from sentence_transformers import datasets

import datasets as dts
from datasets import load_dataset

from torch.utils.data import DataLoader

In [ ]:
# Load and mount the drive helper
from google.colab import drive

# This will prompt for authorization
drive.mount('/content/drive')

# Change the working directory
%cd /content/drive/MyDrive/tsdae

# Create a path variable
file_path = "/content/drive/MyDrive/tsdae"

## Load and Prepare the Pre-Training arXiv Data

We are going to use part of [Kaggle arXiv Dataset](https://www.kaggle.com/datasets/Cornell-University/arxiv), comprised of more than 1.7M scholarly STEM papers and their metadata from the established electronic preprint online platform [arXiv](https://arxiv.org). After downloading the dataset, we extract the preprints with the chosen topics. The full dataset is quite large and we have to adjust our approach to extract just parts of it.


In [ ]:
# Extract the papers with subject "math"
# Skip this step if you are using the math papers datafile

def extract_entries_with_math(filename: str):
    """
    Function to extract those entries that contain the string 'math' in the 'id'.
    """

    # Initialize an empty list to store the extracted entries.
    entries_with_math = []

    with open(filename, 'r') as f:
        for line in f:
            try:
                # Load the JSON object from the line
                data = json.loads(line)

                # Check if the "id" key exists and if it contains "math"
                if "id" in data and "math" in data["id"]:
                    entries_with_math.append(data)

            except json.JSONDecodeError:
                # Print an error message if this line isn't valid JSON
                print(f"Couldn't parse: {line}")

    return entries_with_math

# Snapshot of the arXiv dataset
arxiv_full_dataset = file_path + "/data/arxiv-metadata-oai-snapshot.json"

# Extract the mathematics papers
entries = extract_entries_with_math(arxiv_full_dataset)

# Save the dataset as a JSON object
arxiv_dataset_math = file_path + "/data/arxiv_math_dataset.json"

with open(arxiv_dataset_math, 'w') as fout:
    json.dump(entries, fout)

In [ ]:
# Load the json file containing the selected publications
filename = file_path+"/data/arxiv_math_dataset.json"

with open(filename, 'r') as fin:
        dataset = json.load(fin)

In [ ]:
# Read the data as a Pandas dataframe
df = pd.DataFrame(dataset)

# take a look at the data
df.head(2)

In [ ]:
# Parse the titles by transforming the LaTeX script

parsed_titles = []

for i,a in df.iterrows():
    """
    Function to replace LaTeX script with ISO code.
    """
    # Parse titles
    try:
        parsed_titles.append(LatexNodes2Text().latex_to_text(a['title']).replace('\n', ' ').strip())
    except:
        parsed_titles.append(a['title'].replace('\n', ' ').strip())

df['parsed_title'] = parsed_titles

In [ ]:
# Extract the titles as a list
train_sentences = df.parsed_title.to_list()
# The size of the dataset
len(train_sentences)

In [ ]:
# Create the special denoising dataset that adds noise to the data
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

In [ ]:
# Print an example
print(train_dataset[2010])

In [ ]:
# Batch data loader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)

## Model & Pre-Training

In [ ]:
# Choose a model to build the word embeddings
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)

# Choose the pooling method for the word embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')

# Build the sentence transformer using the two modules
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
# Use the denoising auto-encoder loss and tie encoder-decoder weights
train_loss = losses.DenoisingAutoEncoderLoss(model,
                                             decoder_name_or_path=model_name,
                                             tie_encoder_decoder=True)

In [ ]:
# Start the clock
start_time = time.time()

# Call the fit method to train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True,
    use_amp=True # set to False if GPU does not support FP16 cores
)

# Stop the clock
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Model trained in {elapsed_time:.2f} seconds, on a Google Colab Pro with A100 GPU & High-RAM.")

# Save path of the model
pretrained_model_save_path = 'output/tsdae-bert-uncased-math'
# Save the model locally
model.save(pretrained_model_save_path)

## Evaluate TSDAE Pre-Trained Model

Perform standard evaluation on benchmark STS datasets. These evaluations however are not using the specific domain we are working with and might not reflect the true performance of the model.

In [ ]:
# Save path of the model
pre_model_save_path = 'output/tsdae-bert-uncased-math'
# Load the model if necessary
model = SentenceTransformer(pre_model_save_path)

In [ ]:
# Import the STS benchmark dataset from HuggingFace
sts = dts.load_dataset('glue', 'stsb', split='validation')

# Take a peek at the dataset
sts

In [ ]:
# Take a peek at one of the entries
sts['idx'][100], sts['sentence1'][100], sts['sentence2'][100], sts['label'][100]

In [ ]:
# Normalize the [0, 5] range to [0, 1]
sts = sts.map(lambda x: {'label': x['label'] / 5.0})

# Create a list to store the parsed data
samples = []

for sample in sts:
    # Reformat to use InputExample
    samples.append(InputExample(
        texts=[sample['sentence1'], sample['sentence2']],
        label=sample['label']
    ))

# Instantiate the evaluation module
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    samples, write_csv=False
)

In [ ]:
# Score estimation of the pre-trained model
ev_tsdae = evaluator(model)
print(f"The score for the TSDAE pre-trained model is: {ev_tsdae}.")

In [ ]:
# Score estimation for the baseline model with CLS pooling
bert = models.Transformer('bert-base-uncased')
pooling = models.Pooling(bert.get_word_embedding_dimension(), 'cls')

bert_model = SentenceTransformer(modules=[bert, pooling])
ev_bert_base_uncased = evaluator(bert_model)
print(f"The score for the baseline model is: {ev_bert_base_uncased}.")

In [ ]:
# Score estimation on a full scope advanced pretrained model
all_model = SentenceTransformer('all-mpnet-base-v2')
ev_all_mpnet_base = evaluator(all_model)
print(f"The score for the all-mpnet-base-v2 is: {ev_all_mpnet_base}.")

In [ ]:
# Score estimation on a second full scope more advanced pretrained model
# This model was tuned for semantic search
qa_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
ev_multi_qa_mpnet_base = evaluator(qa_model)
print(f"The score for the multi-qa-mpnet-base-dot-v1 model is: {ev_multi_qa_mpnet_base}.")

## Fine-Tuning on AllNLI Dataset

### Download and Prepare the Datasets

This section is based on [sentence-transformers/examples/training/nli/training_nli_v2.py](https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/nli/training_nli_v2.py).

In [ ]:
# Check if dataset exist. If not, download and extract  it
nli_dataset_path = 'data/AllNLI.tsv.gz'
sts_dataset_path = 'data/stsbenchmark.tsv.gz'

if not os.path.exists(nli_dataset_path):
    util.http_get('https://sbert.net/datasets/AllNLI.tsv.gz', nli_dataset_path)

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

In [ ]:
# Parse the dataset
def add_to_samples(sent1, sent2, label):
    if sent1 not in train_data:
        train_data[sent1] = {'contradiction': set(), 'entailment': set(), 'neutral': set()}
    train_data[sent1][label].add(sent2)

train_data = {}
with gzip.open(nli_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['split'] == 'train':
            sent1 = row['sentence1'].strip()
            sent2 = row['sentence2'].strip()

            add_to_samples(sent1, sent2, row['label'])
            add_to_samples(sent2, sent1, row['label'])  # Also add the opposite

train_samples = []
for sent1, others in train_data.items():
    if len(others['entailment']) > 0 and len(others['contradiction']) > 0:
        train_samples.append(InputExample(texts=[sent1, random.choice(list(others['entailment'])), random.choice(list(others['contradiction']))]))
        train_samples.append(InputExample(texts=[random.choice(list(others['entailment'])), sent1, random.choice(list(others['contradiction']))]))


In [ ]:
# Determine the size of the finetuning dataset
len(train_samples)

In [ ]:
# Special data loader that avoid duplicates within a batch
train_dataloader = datasets.NoDuplicatesDataLoader(train_samples,
                                                   batch_size=32) # 128 default, use smaller batch to fit on Colab

### Fine-Tune the TSDAE Pretrained Model

The system trains BERT (or any other transformer model like RoBERTa, DistilBERT etc.) on the SNLI + MultiNLI (AllNLI) dataset
with MultipleNegativesRankingLoss. Entailments are positive pairs and the contradiction on AllNLI dataset is added as a hard negative.
At every 10% training steps, the model is evaluated on the STS benchmark dataset

In [ ]:
# Set the model parameters
model_name = 'output/tsdae-bert-uncased-math'
train_batch_size = 32 # The larger, the better the results (usually), but requires more GPU memory
max_seq_length = 75
num_epochs = 1


In [ ]:
# Load the local model
local_model = SentenceTransformer(model_name)

In [ ]:
# Our training loss
train_loss = losses.MultipleNegativesRankingLoss(local_model)

In [ ]:

# Read STS benchmark dataset and use it as evaluation set
dev_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['split'] == 'dev':
            score = float(row['score']) / 5.0 #Normalize score to range 0 ... 1
            dev_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))

dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples,
                                                                 batch_size=train_batch_size,
                                                                 name='sts-dev')


In [ ]:
# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up

# Path to save the finetuned model
model_save_path = 'output/finetuned-bert-uncased-math'

# Start the clock
start_time = time.time()

# Train the model
local_model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=int(len(train_dataloader)*0.1),
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          use_amp=True          # Set to True, if your GPU supports FP16 operations
          )

# Stop the clock
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Model finetuned in {elapsed_time:.2f} seconds, on a Google Colab Pro with A100 GPU & High-RAM.")


In [ ]:
# Score estimation of the finetuned model
ev_finetuned = evaluator(local_model)
print(f"The score for the finetuned model is: {ev_finetuned}.")